In [51]:
import pandas as pd
import re
import numpy as np
from Bio import SeqIO, SeqRecord

In [52]:
# Load the new stranded bedgraph file
data = pd.read_csv('repeat_mapping_coverage_filtered.tdf.bedgraph', sep='\t', header=None)
data = data.drop(columns=[0, 1])  # Remove chromosome and start position columns

In [53]:
colnames = ['Pos', 'Positive Strand A', 'Positive Strand C', 'Positive Strand G', 'Positive Strand T', 'Positive Strand N', 'Positive Strand DEL', 'Positive Strand INS', 'Negative Strand A', 'Negative Strand C', 'Negative Strand G', 'Negative Strand T', 'Negative Strand N', 'Negative Strand DEL', 'Negative Strand INS']

In [54]:
# Set column names and calculate basic strand metrics
data.columns = colnames

# Calculate total coverage per strand
data['pos_strand_total'] = (data['Positive Strand A'] + data['Positive Strand C'] + 
                           data['Positive Strand G'] + data['Positive Strand T'])
data['neg_strand_total'] = (data['Negative Strand A'] + data['Negative Strand C'] + 
                           data['Negative Strand G'] + data['Negative Strand T'])
data['total_coverage'] = data['pos_strand_total'] + data['neg_strand_total']

# Calculate strand balance (0.5 = perfectly balanced)
data['strand_balance'] = np.where(
    data['total_coverage'] > 0,
    data['pos_strand_total'] / data['total_coverage'],
    0.5
)

print(f"Data loaded: {len(data)} positions")
print(f"Mean strand balance: {data['strand_balance'].mean():.3f}")
print(f"Positions with severely imbalanced strands (<0.2 or >0.8): {((data['strand_balance'] < 0.2) | (data['strand_balance'] > 0.8)).sum()}")

Data loaded: 7493 positions
Mean strand balance: 0.494
Positions with severely imbalanced strands (<0.2 or >0.8): 0


In [55]:
# Import numpy for calculations
import numpy as np
from scipy import stats

# Infer reference base from most common base per position
print("Inferring reference bases from most common base per position...")
ref_bases = []
for _, row in data.iterrows():
    bases = ['A', 'C', 'G', 'T']
    counts = [row[f'Positive Strand {b}'] + row[f'Negative Strand {b}'] for b in bases]
    max_idx = np.argmax(counts)
    ref_bases.append(bases[max_idx])

data['ref'] = ref_bases
print("Reference bases inferred successfully")

Inferring reference bases from most common base per position...
Reference bases inferred successfully


In [56]:
# Strand-aware SNP analysis function
def analyze_strand_variants(df):
    """
    Analyze variants with strand bias considerations
    """
    result_df = df.copy()
    bases = ['A', 'C', 'G', 'T']
    
    # Initialize result columns
    result_df['pos_strand_alt_base'] = 'N'
    result_df['neg_strand_alt_base'] = 'N'
    result_df['pos_strand_alt_count'] = 0
    result_df['neg_strand_alt_count'] = 0
    result_df['pos_strand_alt_freq'] = 0.0
    result_df['neg_strand_alt_freq'] = 0.0
    result_df['pos_strand_ref_count'] = 0
    result_df['neg_strand_ref_count'] = 0
    result_df['pos_strand_ref_freq'] = 0.0
    result_df['neg_strand_ref_freq'] = 0.0
    result_df['strand_bias_pvalue'] = 1.0
    
    print("Calculating strand-specific variant metrics...")
    
    for idx, row in result_df.iterrows():
        if idx % 1000 == 0:
            print(f"Processing position {idx}/{len(result_df)}")
            
        ref_base = row['ref']
        
        # Calculate for positive strand
        pos_counts = {base: row[f'Positive Strand {base}'] for base in bases}
        pos_total = row['pos_strand_total']
        
        if pos_total > 0:
            # Reference counts and frequency
            result_df.at[idx, 'pos_strand_ref_count'] = pos_counts.get(ref_base, 0)
            result_df.at[idx, 'pos_strand_ref_freq'] = pos_counts.get(ref_base, 0) / pos_total
            
            # Find most common alternative base
            alt_counts = {base: count for base, count in pos_counts.items() if base != ref_base}
            if alt_counts:
                max_alt_base = max(alt_counts, key=alt_counts.get)
                max_alt_count = alt_counts[max_alt_base]
                result_df.at[idx, 'pos_strand_alt_base'] = max_alt_base
                result_df.at[idx, 'pos_strand_alt_count'] = max_alt_count
                result_df.at[idx, 'pos_strand_alt_freq'] = max_alt_count / pos_total
        
        # Calculate for negative strand
        neg_counts = {base: row[f'Negative Strand {base}'] for base in bases}
        neg_total = row['neg_strand_total']
        
        if neg_total > 0:
            # Reference counts and frequency
            result_df.at[idx, 'neg_strand_ref_count'] = neg_counts.get(ref_base, 0)
            result_df.at[idx, 'neg_strand_ref_freq'] = neg_counts.get(ref_base, 0) / neg_total
            
            # Find most common alternative base
            alt_counts = {base: count for base, count in neg_counts.items() if base != ref_base}
            if alt_counts:
                max_alt_base = max(alt_counts, key=alt_counts.get)
                max_alt_count = alt_counts[max_alt_base]
                result_df.at[idx, 'neg_strand_alt_base'] = max_alt_base
                result_df.at[idx, 'neg_strand_alt_count'] = max_alt_count
                result_df.at[idx, 'neg_strand_alt_freq'] = max_alt_count / neg_total
        
        # Calculate strand bias using Fisher's exact test
        pos_alt = result_df.at[idx, 'pos_strand_alt_count']
        pos_ref = result_df.at[idx, 'pos_strand_ref_count']
        neg_alt = result_df.at[idx, 'neg_strand_alt_count']
        neg_ref = result_df.at[idx, 'neg_strand_ref_count']
        
        # Only calculate if we have variants and both strands have coverage
        if (pos_alt + neg_alt > 0) and (pos_alt + pos_ref > 0) and (neg_alt + neg_ref > 0):
            try:
                contingency = np.array([[pos_alt, pos_ref], [neg_alt, neg_ref]])
                _, p_value = stats.fisher_exact(contingency)
                result_df.at[idx, 'strand_bias_pvalue'] = p_value
            except:
                # Fallback calculation if Fisher's test fails
                pos_freq = pos_alt / (pos_alt + pos_ref)
                neg_freq = neg_alt / (neg_alt + neg_ref)
                # Simple difference-based score (higher = less biased)
                result_df.at[idx, 'strand_bias_pvalue'] = 1 - abs(pos_freq - neg_freq)
    
    # Calculate combined metrics
    result_df['combined_alt_count'] = result_df['pos_strand_alt_count'] + result_df['neg_strand_alt_count']
    result_df['combined_alt_freq'] = result_df['combined_alt_count'] / result_df['total_coverage']
    result_df['combined_ref_freq'] = (result_df['pos_strand_ref_count'] + result_df['neg_strand_ref_count']) / result_df['total_coverage']
    
    # Check if alternative bases are consistent across strands
    result_df['consistent_alt_base'] = result_df['pos_strand_alt_base'] == result_df['neg_strand_alt_base']
    
    return result_df

# Run the strand analysis
print("Starting strand-aware analysis...")
strand_data = analyze_strand_variants(data)

Starting strand-aware analysis...
Calculating strand-specific variant metrics...
Processing position 0/7493
Processing position 1000/7493
Processing position 2000/7493
Processing position 3000/7493
Processing position 4000/7493
Processing position 5000/7493
Processing position 6000/7493
Processing position 7000/7493


In [57]:
# Strand-aware SNP filtering
def identify_genuine_snps_strand_aware(df, min_coverage=0, min_alt_freq=0.0075, 
                                     min_strand_bias_p=0.05, min_alt_reads_per_strand=5):
    """
    Identify genuine SNPs using strand-aware criteria
    """
    result_df = df.copy()
    
    # Apply filters
    result_df['sufficient_coverage'] = result_df['total_coverage'] >= min_coverage
    result_df['sufficient_alt_freq'] = result_df['combined_alt_freq'] >= min_alt_freq
    result_df['low_strand_bias'] = result_df['strand_bias_pvalue'] >= min_strand_bias_p
    result_df['consistent_variant'] = result_df['consistent_alt_base']
    
    # Both strands must have minimum alternative reads
    result_df['sufficient_alt_reads_both_strands'] = (
        (result_df['pos_strand_alt_count'] >= min_alt_reads_per_strand) & 
        (result_df['neg_strand_alt_count'] >= min_alt_reads_per_strand)
    )
    
    # Balanced strand coverage (neither strand dominates completely)
    result_df['balanced_strands'] = (result_df['strand_balance'] >= 0.1) & (result_df['strand_balance'] <= 0.9)
    
    # Combined SNP calling
    result_df['candidate_snp'] = (
        result_df['sufficient_coverage'] & 
        result_df['sufficient_alt_freq'] & 
        result_df['low_strand_bias'] & 
        result_df['consistent_variant'] & 
        result_df['balanced_strands']
    )
    
    # High-confidence SNPs (stricter criteria)
    result_df['high_confidence_snp'] = (
        result_df['candidate_snp'] & 
        result_df['sufficient_alt_reads_both_strands'] & 
        (result_df['combined_alt_freq'] >= 0.01) &
        (result_df['strand_bias_pvalue'] >= 0.1) &
        (result_df['total_coverage'] >= 200)
    )
    
    return result_df

# Apply strand-aware SNP filtering
print("Applying strand-aware SNP filters...")
final_results = identify_genuine_snps_strand_aware(strand_data)

# Generate summary
print("\\n" + "="*60)
print("STRAND-AWARE SNP ANALYSIS SUMMARY")
print("="*60)

print(f"Total positions analyzed: {len(final_results):,}")
print(f"Positions with sufficient coverage (≥100x): {final_results['sufficient_coverage'].sum():,}")
print(f"Positions with variants (≥1%): {final_results['sufficient_alt_freq'].sum():,}")
print(f"Positions with low strand bias (p≥0.05): {final_results['low_strand_bias'].sum():,}")
print(f"Positions with consistent variants across strands: {final_results['consistent_variant'].sum():,}")
print(f"Positions with balanced strand coverage: {final_results['balanced_strands'].sum():,}")
print(f"Positions with sufficient alt reads on both strands: {final_results['sufficient_alt_reads_both_strands'].sum():,}")
print(f"Candidate SNPs: {final_results['candidate_snp'].sum():,}")
print(f"High-confidence SNPs: {final_results['high_confidence_snp'].sum():,}")

print("\\nStrand bias distribution:")
print(f"Strong bias (p < 0.01): {(final_results['strand_bias_pvalue'] < 0.01).sum():,}")
print(f"Moderate bias (0.01 ≤ p < 0.05): {((final_results['strand_bias_pvalue'] >= 0.01) & (final_results['strand_bias_pvalue'] < 0.05)).sum():,}")
print(f"Low bias (p ≥ 0.05): {(final_results['strand_bias_pvalue'] >= 0.05).sum():,}")

Applying strand-aware SNP filters...
\n============================================================
STRAND-AWARE SNP ANALYSIS SUMMARY
Total positions analyzed: 7,493
Positions with sufficient coverage (≥100x): 7,493
Positions with variants (≥1%): 951
Positions with low strand bias (p≥0.05): 5,594
Positions with consistent variants across strands: 2,825
Positions with balanced strand coverage: 7,493
Positions with sufficient alt reads on both strands: 828
Candidate SNPs: 95
High-confidence SNPs: 26
\nStrand bias distribution:
Strong bias (p < 0.01): 1,169
Moderate bias (0.01 ≤ p < 0.05): 730
Low bias (p ≥ 0.05): 5,594


In [58]:
final_results['candidate_snp'].sum()

np.int64(95)

In [59]:
# Display top candidate SNPs
if final_results['candidate_snp'].sum() > 0:
    print("\n" + "="*50)
    print("TOP CANDIDATE SNPs")
    print("="*50)
    
    candidates = final_results[final_results['candidate_snp']].copy()
    candidates = candidates.sort_values('combined_alt_freq', ascending=False)
    
    display_cols = ['Pos', 'ref', 'pos_strand_alt_base', 'neg_strand_alt_base',
                   'total_coverage', 'combined_alt_freq', 'strand_bias_pvalue',
                   'pos_strand_alt_count', 'neg_strand_alt_count', 'strand_balance']
    
    print("Key:")
    print("- Pos: Position in sequence")
    print("- ref: Reference base")
    print("- pos/neg_strand_alt_base: Alternative base on each strand")
    print("- combined_alt_freq: Overall alternative allele frequency")
    print("- strand_bias_pvalue: Fisher's exact test p-value (higher = less biased)")
    print("- strand_balance: Positive strand fraction (0.5 = balanced)")
    print()
    
    print(candidates[display_cols].head(20))
    
    # Save candidate SNPs
    candidates[display_cols].to_csv('strand_aware_candidate_snps.tsv', sep='\t', index=False)
    print(f"\nCandidate SNPs saved to: strand_aware_candidate_snps.tsv")

# Display high-confidence SNPs
if final_results['high_confidence_snp'].sum() > 0:
    print("\n" + "="*50)
    print("HIGH-CONFIDENCE SNPs")
    print("="*50)
    
    high_conf = final_results[final_results['high_confidence_snp']].copy()
    high_conf = high_conf.sort_values('combined_alt_freq', ascending=False)
    
    print(high_conf[display_cols])
    
    # Save high-confidence SNPs
    high_conf[display_cols].to_csv('strand_aware_high_confidence_snps.tsv', sep='\t', index=False)
    print(f"\nHigh-confidence SNPs saved to: strand_aware_high_confidence_snps.tsv")
else:
    print("\nNo high-confidence SNPs identified with current criteria")

# Show examples of strand-biased variants (likely artifacts)
strand_biased = final_results[
    (final_results['combined_alt_freq'] >= 0.01) & 
    (final_results['strand_bias_pvalue'] < 0.01)
].copy()

if len(strand_biased) > 0:
    print("\n" + "="*50)
    print("EXAMPLES OF STRAND-BIASED VARIANTS (Likely Artifacts)")
    print("="*50)
    
    strand_biased = strand_biased.sort_values('combined_alt_freq', ascending=False)
    print(strand_biased[display_cols].head(10))
    print(f"\nTotal strand-biased variants: {len(strand_biased)}")

# Summary recommendations
print("\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)
print("1. Focus on high-confidence SNPs for validation")
print("2. Candidate SNPs require additional validation:")
print("   - PCR amplification and Sanger sequencing")
print("   - Independent sequencing technology")
print("   - Single-molecule analysis")
print("3. Positions with strong strand bias (p < 0.01) are likely artifacts")
print("4. Consider the biological context - rRNA is highly conserved")
print("5. Validate any findings in multiple biological replicates")


TOP CANDIDATE SNPs
Key:
- Pos: Position in sequence
- ref: Reference base
- pos/neg_strand_alt_base: Alternative base on each strand
- combined_alt_freq: Overall alternative allele frequency
- strand_bias_pvalue: Fisher's exact test p-value (higher = less biased)
- strand_balance: Positive strand fraction (0.5 = balanced)

       Pos ref pos_strand_alt_base neg_strand_alt_base  total_coverage  \
256    257   A                   T                   T          2228.0   
2700  2701   C                   G                   G          2220.0   
6160  6161   T                   C                   C          2286.0   
4110  4111   C                   G                   G          2286.0   
4234  4235   C                   T                   T          2292.0   
5873  5874   A                   G                   G          2232.0   
5811  5812   G                   C                   C          2307.0   
2635  2636   G                   C                   C          2222.0   
6721  67

In [60]:
candidates.sort_values(by = 'Pos')

,Pos,Positive Strand A,Positive Strand C,Positive Strand G,Positive Strand T,Positive Strand N,Positive Strand DEL,Positive Strand INS,Negative Strand A,Negative Strand C,...,combined_ref_freq,consistent_alt_base,sufficient_coverage,sufficient_alt_freq,low_strand_bias,consistent_variant,sufficient_alt_reads_both_strands,balanced_strands,candidate_snp,high_confidence_snp
183,184,3.0,1069.0,7.0,8.0,0.0,18.0,5.0,1.0,1060.0,...,0.980203,True,True,True,True,True,True,True,True,True
255,256,1096.0,0.0,11.0,1.0,0.0,3.0,1.0,1111.0,0.0,...,0.989686,True,True,True,True,True,True,True,True,False
256,257,1080.0,0.0,0.0,28.0,0.0,3.0,12.0,1099.0,0.0,...,0.978007,True,True,True,True,True,True,True,True,True
384,385,7.0,1086.0,11.0,5.0,0.0,4.0,14.0,1.0,1118.0,...,0.984368,True,True,True,True,True,True,True,True,False
532,533,1090.0,5.0,13.0,2.0,0.0,9.0,3.0,1127.0,2.0,...,0.987968,True,True,True,True,True,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7201,7202,0.0,5.0,4.0,1103.0,0.0,7.0,0.0,0.0,12.0,...,0.988918,True,True,True,True,True,True,True,True,False
7248,7249,1102.0,2.0,11.0,0.0,0.0,4.0,2.0,1124.0,1.0,...,0.988455,True,True,True,True,True,True,True,True,False
7300,7301,11.0,0.0,1101.0,1.0,0.0,4.0,3.0,6.0,0.0,...,0.992028,True,True,True,True,True,True,True,True,False
7352,7353,7.0,1079.0,12.0,1.0,0.0,16.0,7.0,0.0,1136.0,...,0.987517,True,True,True,True,True,True,True,True,False


In [61]:
# repeat_path = 'repeat_records_revcomp_filtered.fasta'
repeat_path = 'repeat_records_filtered.fasta'
# mappings_path = 'repeat_records_revcomp_filtered_mapped_to_AF16_45S_rDNA.sorted.paf'
mappings_path = 'repeat_records_filtered_mapped_to_AF16_45S_rDNA.sorted.paf'
# mappings_path_bam = 'repeat_records_revcomp_filtered_mapped_to_AF16_45S_rDNA.sorted.bam'
mappings_path_bam = 'repeat_records_filtered_mapped_to_AF16_45S_rDNA.sorted.bam'

In [62]:
repeats = list(SeqIO.parse(repeat_path, 'fasta'))
repeat_ids = [repeat.id for repeat in repeats]
repeat_seqs = [repeat.seq for repeat in repeats]
repeat_mappings = pd.read_csv(mappings_path, sep='\t', header=None)

In [63]:
sorted_seqs = []

for item in repeat_mappings.iterrows():
    sorted_seqs.append(repeat_seqs[repeat_ids.index(item[1][0])])


In [64]:
repeat_mappings['Seq'] = sorted_seqs

In [65]:
import pysam

# Open BAM file (read-only, indexed BAM recommended)
bamfile = pysam.AlignmentFile(mappings_path_bam, "rb")
fixed_strings = []
for index, row in repeat_mappings.iterrows():
    read_id = row[0]
    #print(f"Fixing CIGAR string for {index}: {read_id}")

    # Fetch read by name (uses BAM index)
    for read in bamfile.fetch(until_eof=True):
        if read.query_name == read_id:
            fixed_string = read.cigarstring
            fixed_strings.append(fixed_string)
            #print(fixed_string)
            break
    else:
        print(f"Read {read_id} not found.")


In [66]:
# Build BAM-derived strand and CIGAR for each read to avoid PAF strand ambiguity
import pysam
bam = pysam.AlignmentFile(mappings_path_bam, "rb")
bam_info = {}
for read in bam.fetch(until_eof=True):
    bam_info[read.query_name] = {
        'cigar': read.cigarstring,
        'bam_strand': '-' if read.is_reverse else '+'
    }

# Map onto repeat_mappings
repeat_mappings['bam_strand'] = repeat_mappings[0].map(lambda rid: bam_info.get(rid, {}).get('bam_strand'))
repeat_mappings['bam_cigar'] = repeat_mappings[0].map(lambda rid: bam_info.get(rid, {}).get('cigar'))

# Prefer BAM CIGAR when available
use_bam_cigar = repeat_mappings['bam_cigar'].notna()
repeat_mappings.loc[use_bam_cigar, 17] = repeat_mappings.loc[use_bam_cigar, 'bam_cigar']

print("BAM strand/cigar columns added. Example:")
print(repeat_mappings[['bam_strand', 4, 17, 'bam_cigar']].head())

BAM strand/cigar columns added. Example:
  bam_strand  4                                                 17  \
0          +  +  71M1D175M1I19M1I347M1I5M2D84M2I3M2D2M1I173M1D6...   
1          +  +  308M2D103M1D157M1I47M4D744M2I433M6D295M1D60M1I...   
2          +  +  308M2D251M1D81M2I701M6D9M2I302M32D80M3D654M1D2...   
3          +  +  308M2D258M1I48M1D94M1I1M2D11M1I1M2D20M1I112M2D...   
4          +  +  308M3D83M1D3M1D2M2D10M1D164M1D17M6D79M4I1459M1...   

                                           bam_cigar  
0  71M1D175M1I19M1I347M1I5M2D84M2I3M2D2M1I173M1D6...  
1  308M2D103M1D157M1I47M4D744M2I433M6D295M1D60M1I...  
2  308M2D251M1D81M2I701M6D9M2I302M32D80M3D654M1D2...  
3  308M2D258M1I48M1D94M1I1M2D11M1I1M2D20M1I112M2D...  
4  308M3D83M1D3M1D2M2D10M1D164M1D17M6D79M4I1459M1...  


In [67]:
# Inspect repeat_mappings to determine column names/positions we need
cols = list(repeat_mappings.columns)
print(f"Columns ({len(cols)}):\n", cols)

# Show the 17th column (index 16) which should be CIGAR strings
cigar_col = cols[16] if len(cols) > 16 else None
print("\nCIGAR column (17th):", cigar_col)

# Try to guess standard PAF fields
def find_col(candidates):
    for c in candidates:
        if c in repeat_mappings.columns:
            return c
    return None

strand_col = find_col(['strand','Strand','orientation','dir'])
ref_start_col = find_col(['tStart','target_start','t_start','ref_start','RefStart','t_st'])
ref_end_col = find_col(['tEnd','target_end','t_end','ref_end','RefEnd','t_en'])
qry_start_col = find_col(['qStart','query_start','q_start','QueryStart'])
qry_end_col = find_col(['qEnd','query_end','q_end','QueryEnd'])
ref_name_col = find_col(['tName','target','ref','rname','RefName','target_name'])
qry_name_col = find_col(['qName','query','read','qname','ReadName','query_name'])

print("Guessed columns:")
print("  strand:", strand_col)
print("  ref_start:", ref_start_col, "ref_end:", ref_end_col)
print("  qry_start:", qry_start_col, "qry_end:", qry_end_col)
print("  ref_name:", ref_name_col, "qry_name:", qry_name_col)

# Peek at one row
print("\nSample row (first non-empty):")
print(repeat_mappings.head(1).T)

Columns (21):
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 'Seq', 'bam_strand', 'bam_cigar']

CIGAR column (17th): 16
Guessed columns:
  strand: None
  ref_start: None ref_end: None
  qry_start: None qry_end: None
  ref_name: None qry_name: None

Sample row (first non-empty):
                                                            0
0                      00725291-da2c-4ba2-99a8-606106f0ca2b_1
1                                                        7476
2                                                           0
3                                                        7476
4                                                           +
5                                         C_briggsae_45S_rDNA
6                                                        7493
7                                                           0
8                                                        7493
9                                                        7439
10                    

## Strict SNP presence mapping (alt-match only) and strand-aware summary
This section rebuilds the repeat-unit SNP presence using only exact matches to the inferred alternative allele per position and provides per-strand counts to match IGV expectations.

In [68]:
# Recompute presence STRICT with optional CONTEXT:
# - If USE_FLANK_CONTEXT is True: present only if flanks match REF within k mismatches AND center == ALT (strand-aware).
# - If USE_FLANK_CONTEXT is False: present only if center == ALT (strand-aware), ignoring flanks (previous behavior).
import pandas as pd
import numpy as np
from Bio import SeqIO

# Toggle and tunable parameters
USE_FLANK_CONTEXT = True  # set True to enforce flanks; False to ignore flanks (ALT-only at center)
FLANK = 5                  # number of nucleotides on each side to check (when context is used)
MAX_FLANK_MISMATCHES = 1   # allow up to k mismatches across both flanks (when context is used)

# 0) Load reference sequence used for mapping (AF16 45S rDNA), fallback to C_briggsae if needed
ref_seq = None
for ref_fa in ['AF16_45S_rDNA.fasta', 'C_briggsae_45S_rDNA.fasta']:
    try:
        rec = next(SeqIO.parse(ref_fa, 'fasta'))
        ref_seq = str(rec.seq).upper()
        print(f"Loaded reference: {ref_fa} (length={len(ref_seq)})")
        break
    except Exception:
        pass
if ref_seq is None:
    raise RuntimeError("Could not load reference FASTA (AF16_45S_rDNA.fasta or C_briggsae_45S_rDNA.fasta)")

# 1) Determine SNP positions and ALT from earlier results (prefer candidate/high-confidence with known alt)
alt_source = None
for p in ['strand_aware_candidate_snps.tsv', 'strand_aware_high_confidence_snps.tsv']:
    try:
        tmp = pd.read_csv(p, sep='\t')
        if 'Pos' in tmp.columns:
            alt_source = tmp
            break
    except Exception:
        pass

if alt_source is None:
    # If no ALT file, infer ALT as dominant non-ref base from earlier final_results
    alt_rows = final_results[final_results['candidate_snp']].copy() if 'final_results' in globals() else pd.DataFrame()
    if len(alt_rows) == 0:
        raise RuntimeError("No alt-allele table available. Please run earlier SNP calling cells or provide TSV with Pos/ref/alt.")
    alt_source = alt_rows[['Pos','ref','pos_strand_alt_base','neg_strand_alt_base']].copy()
    alt_source['alt'] = np.where(
        alt_source['pos_strand_alt_base'] == alt_source['neg_strand_alt_base'],
        alt_source['pos_strand_alt_base'],
        alt_rows['pos_strand_alt_base']
    )

# Standardize columns and clean
if 'alt' not in alt_source.columns:
    if 'pos_strand_alt_base' in alt_source.columns and 'neg_strand_alt_base' in alt_source.columns:
        alt_source['alt'] = np.where(
            alt_source['pos_strand_alt_base'] == alt_source['neg_strand_alt_base'],
            alt_source['pos_strand_alt_base'],
            alt_source['pos_strand_alt_base']
        )
    else:
        raise RuntimeError("ALT allele not found. Ensure SNP table has 'alt' or pos/neg strand alt columns.")

alt_table = alt_source[['Pos','ref','alt']].dropna(subset=['Pos','alt']).copy()
alt_table['Pos'] = alt_table['Pos'].astype(int)
alt_table['ref'] = alt_table['ref'].astype(str).str.upper()
alt_table['alt'] = alt_table['alt'].astype(str).str.upper()

# 2) Helpers
_comp = str.maketrans({'A':'T','C':'G','G':'C','T':'A','a':'t','c':'g','g':'c','t':'a','N':'N','n':'n'})

def revcomp_base(b):
    if not isinstance(b, str) or len(b) == 0:
        return b
    return b[0].translate(_comp)

def parse_cigar(cigar):
    ops, num = [], ''
    for ch in str(cigar):
        if ch.isdigit():
            num += ch
        else:
            if not num:
                raise ValueError(f"Malformed CIGAR near '{cigar}'")
            ops.append((ch, int(num)))
            num = ''
    return ops

def to_seq_str(x):
    if isinstance(x, str):
        return x
    if isinstance(x, (list, tuple)):
        return ''.join(map(str, x))
    return str(x)

def count_mismatches(a: str, b: str) -> int:
    if len(a) != len(b):
        L = max(len(a), len(b))
        a = (a + 'N'*L)[:L]
        b = (b + 'N'*L)[:L]
    return sum(1 for x, y in zip(a, b) if x != y)

# Build mapping column indices (PAF-like)
QNAME, QLEN, QSTART, QEND, STRAND, TNAME, TLEN, TSTART, TEND = 0,1,2,3,4,5,6,7,8
CIGAR_IDX = 17
SEQ_COL = 'Seq'

# 3) Strict presence with optional context
strict_results = []

for idx, row in repeat_mappings.iterrows():
    try:
        qname = row[QNAME]
        strand = row['bam_strand'] if 'bam_strand' in repeat_mappings.columns and pd.notna(row['bam_strand']) else row[STRAND]
        t_start = int(row[TSTART])
        t_end = int(row[TEND])
        q_start = int(row[QSTART])
        cigar = row[CIGAR_IDX]
        seq = to_seq_str(row[SEQ_COL])
        if pd.isna(cigar) or pd.isna(strand):
            continue
        ops = parse_cigar(cigar)

        # Build reference->query mapping for all aligned positions
        ref_pos = t_start
        qry_pos = q_start
        ref_to_qry = {}
        for op, length in ops:
            if op in ('M','X','='):
                for i in range(length):
                    ref_to_qry[ref_pos + i] = qry_pos + i
                ref_pos += length
                qry_pos += length
            elif op == 'I':
                qry_pos += length
            elif op in ('D','N'):
                ref_pos += length
            elif op == 'S':
                qry_pos += length

        row_out = {'repeat_unit': qname, 'strand': strand}

        for _, snp in alt_table.iterrows():
            pos1 = int(snp['Pos'])
            refb = str(snp['ref']).upper()
            altb = str(snp['alt']).upper()

            # Always require center ALT match (strand-aware)
            center_ref0 = pos1 - 1
            q_center = ref_to_qry.get(center_ref0)
            if q_center is None or q_center < 0 or q_center >= len(seq):
                row_out[f'SNP_{pos1}'] = 0
                continue
            center_base = seq[q_center]
            if strand == '-':
                center_base = revcomp_base(center_base)
            center_base = str(center_base).upper()
            center_match = (center_base == altb)
            if not center_match:
                row_out[f'SNP_{pos1}'] = 0
                continue

            # Optionally enforce flanking context
            if USE_FLANK_CONTEXT:
                left_start = pos1 - FLANK
                left_end = pos1 - 1
                right_start = pos1 + 1
                right_end = pos1 + FLANK

                # If flanks go out of bounds, consider as failing context
                if left_start < 1 or right_end > len(ref_seq):
                    row_out[f'SNP_{pos1}'] = 0
                    continue

                # Extract reference flanks (plus orientation)
                ref_left = ref_seq[left_start-1:left_end]
                ref_right = ref_seq[right_start-1:right_end]

                if strand == '-':
                    ref_left = ''.join(revcomp_base(b) for b in ref_left[::-1])
                    ref_right = ''.join(revcomp_base(b) for b in ref_right[::-1])
                # Build query flanks via mapping; count missing/discordant as mismatches
                q_left_chars = []
                for rp in range(left_start-1, left_end):
                    q = ref_to_qry.get(rp)
                    if q is None or q < 0 or q >= len(seq):
                        q_left_chars.append('N')
                    else:
                        b = seq[q]
                        if strand == '-':
                            b = revcomp_base(b)
                        q_left_chars.append(str(b).upper())

                q_right_chars = []
                for rp in range(right_start-1, right_end):
                    q = ref_to_qry.get(rp)
                    if q is None or q < 0 or q >= len(seq):
                        q_right_chars.append('N')
                    else:
                        b = seq[q]
                        if strand == '-':
                            b = revcomp_base(b)
                        q_right_chars.append(str(b).upper())

                q_left = ''.join(q_left_chars)
                q_right = ''.join(q_right_chars)
                mismatches = count_mismatches(q_left, ref_left) + count_mismatches(q_right, ref_right)
                flank_match = (mismatches <= MAX_FLANK_MISMATCHES)
            else:
                flank_match = True  # ignore flank context

            row_out[f'SNP_{pos1}'] = int(flank_match and center_match)

        strict_results.append(row_out)
    except Exception:
        pass

strict_df = pd.DataFrame(strict_results)
# Save strict matrix
mode = 'with_context' if USE_FLANK_CONTEXT else 'alt_only'
strict_df.to_csv('repeat_unit_top_snp_presence_strict.tsv', sep='\t', index=False)
print(f"Saved presence matrix ({mode}; FLANK={FLANK}, k={MAX_FLANK_MISMATCHES if USE_FLANK_CONTEXT else 'NA'}): repeat_unit_top_snp_presence_strict.tsv ({len(strict_df)} rows)")

# 4) Summarize by SNP and strand to compare with IGV
summary_rows = []
for snp_pos in sorted(alt_table['Pos'].unique()):
    pos_col = f'SNP_{snp_pos}'
    if pos_col in strict_df.columns:
        pos_df = strict_df[[pos_col,'strand']].copy()
        total = int(pos_df[pos_col].sum())
        pos_count = int(pos_df[(pos_df['strand'] == '+') & (pos_df[pos_col] == 1)].shape[0])
        neg_count = int(pos_df[(pos_df['strand'] == '-') & (pos_df[pos_col] == 1)].shape[0])
        summary_rows.append({'Pos': snp_pos, 'total_present_units': total, 'present_plus': pos_count, 'present_minus': neg_count})

summary_df = pd.DataFrame(summary_rows).sort_values('Pos')
print("\nPresence summary (units meeting criteria):")
print(summary_df)

# Quick check specifically for 257 if present
if 'SNP_257' in strict_df.columns:
    plus = strict_df[(strict_df['strand'] == '+') & (strict_df['SNP_257'] == 1)].shape[0]
    minus = strict_df[(strict_df['strand'] == '-') & (strict_df['SNP_257'] == 1)].shape[0]
    total = strict_df[strict_df['SNP_257'] == 1].shape[0]
    print(f"\nSNP_257 counts ({mode}; FLANK={FLANK}, k={MAX_FLANK_MISMATCHES if USE_FLANK_CONTEXT else 'NA'}) -> total: {total}, plus: {plus}, minus: {minus}")

# Also provide a filtered presence table for quick inspection
if 'SNP_257' in strict_df.columns:
    data_test_strict = strict_df[strict_df['SNP_257'] == 1][['repeat_unit','strand','SNP_257']]
    data_test_strict.to_csv('data_test_SNP_257_strict.tsv', sep='\t', index=False)
    print("Saved subset for SNP_257: data_test_SNP_257_strict.tsv")

Loaded reference: AF16_45S_rDNA.fasta (length=7493)
Saved presence matrix (with_context; FLANK=5, k=1): repeat_unit_top_snp_presence_strict.tsv (2770 rows)

Presence summary (units meeting criteria):
     Pos  total_present_units  present_plus  present_minus
0    184                    5             5              0
1    256                   11            11              0
2    257                   25            25              0
3    385                    8             8              0
4    533                    6             6              0
..   ...                  ...           ...            ...
90  7202                    1             1              0
91  7249                    3             3              0
92  7301                    7             7              0
93  7353                    9             9              0
94  7389                    9             9              0

[95 rows x 4 columns]

SNP_257 counts (with_context; FLANK=5, k=1) -> total: 25, plus: 25, 

In [69]:
data_snps = pd.read_csv("repeat_unit_top_snp_presence_strict.tsv", sep = '\t')

In [70]:
data_snps.shape

(2770, 97)

In [71]:
rel_data = data_snps[data_snps['strand'] == '+']

In [72]:
rel_data.shape


(1389, 97)

In [73]:
rel_data[rel_data['repeat_unit'].str.contains('3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca')]

,repeat_unit,strand,SNP_257,SNP_2701,SNP_6161,SNP_4111,SNP_4235,SNP_5874,SNP_5812,SNP_2636,...,SNP_3050,SNP_1952,SNP_2424,SNP_385,SNP_1898,SNP_1458,SNP_7249,SNP_7202,SNP_7301,SNP_890
234,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_1,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
235,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_2,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
236,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_3,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
237,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_4,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
238,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_5,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
239,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_6,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
240,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
241,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_9,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
242,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_10,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
243,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_11,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
big_groups = [group for _, group in rel_data.groupby([col for col in rel_data.columns if col.startswith('SNP_')]) if len(group) > 1]


In [75]:
groups = list(rel_data.groupby([col for col in rel_data.columns if col.startswith('SNP_')]))

In [76]:
big_groups[1]

,repeat_unit,strand,SNP_257,SNP_2701,SNP_6161,SNP_4111,SNP_4235,SNP_5874,SNP_5812,SNP_2636,...,SNP_3050,SNP_1952,SNP_2424,SNP_385,SNP_1898,SNP_1458,SNP_7249,SNP_7202,SNP_7301,SNP_890
650,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_3,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
812,d084f1bc-c41b-4d41-819d-25b743e1369f_2,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [77]:
repeat_mappings[repeat_mappings[0].str.contains('a0ee7cd6-f4ed-473f-802b-4c1e262d949e')]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,Seq,bam_strand,bam_cigar
648,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_1,7479,0,7479,+,C_briggsae_45S_rDNA,7493,0,7493,7449,...,60,tp:A:P,NM:i:55,mm:i:19,gn:i:36,go:i:24,510M1I56M1D54M2D1M1I5M1I5M1D549M1D1361M2I2M1D2...,"(G, T, T, T, C, C, C, C, T, T, C, T, T, T, T, ...",+,510M1I56M1D54M2D1M1I5M1I5M1D549M1D1361M2I2M1D2...
649,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_2,7497,0,7497,+,C_briggsae_45S_rDNA,7493,0,7493,7432,...,60,tp:A:P,NM:i:82,mm:i:44,gn:i:38,go:i:31,546M1I64M1I151M2D293M1I172M1I7M2D248M1I51M1I26...,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,546M1I64M1I151M2D293M1I172M1I7M2D248M1I51M1I26...
650,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_3,7475,0,7475,+,C_briggsae_45S_rDNA,7493,0,7493,7434,...,60,tp:A:P,NM:i:72,mm:i:28,gn:i:44,go:i:30,308M1D256M2D135M1I185M1I348M1I55M6D627M3D4M2D2...,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,308M1D256M2D135M1I185M1I348M1I55M6D627M3D4M2D2...
651,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_4,7486,0,7486,+,C_briggsae_45S_rDNA,7493,0,7493,7435,...,60,tp:A:P,NM:i:78,mm:i:31,gn:i:47,go:i:37,4M1D303M2D72M1D135M1D133M1D248M1D82M2I2M1D243M...,"(G, T, T, T, C, C, T, T, T, T, T, T, T, T, G, ...",+,4M1D303M2D72M1D135M1D133M1D248M1D82M2I2M1D243M...
652,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_5,7483,0,7483,+,C_briggsae_45S_rDNA,7493,0,7493,7426,...,60,tp:A:P,NM:i:89,mm:i:35,gn:i:54,go:i:41,42M1D1M1D263M1D74M2I180M1I1M1I51M2D458M1I143M2...,"(G, T, T, C, C, C, C, C, T, T, T, T, T, T, T, ...",+,42M1D1M1D263M1D74M2I180M1I1M1I51M2D458M1I143M2...
2229,a0ee7cd6-f4ed-473f-802b-4c1e262d949e_0,7305,0,7305,+,C_briggsae_45S_rDNA,7493,184,7493,7262,...,60,tp:A:P,NM:i:66,mm:i:24,gn:i:42,go:i:33,124M2D13M1D378M1I113M1D115M1D3M1I140M1I464M1I1...,"(G, G, G, C, C, T, C, A, A, A, T, T, T, T, C, ...",+,124M2D13M1D378M1I113M1D115M1D3M1I140M1I464M1I1...


In [78]:
def pick_groups(df):
    rows_with_1 = (df.iloc[:, 2:-1].sum(axis=1) > 0).sum()
    return(rows_with_1 > 1)

In [79]:
rel_data["group_id"] = rel_data["repeat_unit"].str.rsplit("_", n=1).str[0]

/var/folders/z1/qtlv_5t952n4zc8j40z8_xq00000gn/T/ipykernel_63759/2647906927.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rel_data["group_id"] = rel_data["repeat_unit"].str.rsplit("_", n=1).str[0]


In [80]:
name_groupings = list(rel_data.groupby('group_id'))

In [88]:
# useful_reads = [item for item in name_groupings if (pick_groups(item[1]) and item[1].shape[0] > 10)]
useful_reads = [item for item in name_groupings if (pick_groups(item[1]))]

In [89]:
len(useful_reads)

98

In [90]:
idx = 14

In [91]:
for i, item in enumerate(useful_reads[idx][1].itertuples(index=False)):
    if i == 0:
        print(item[0])
    print(i, sum(item[2:-1]) > 0)

2903071d-6bb6-4abf-b49d-64dc73d7a01a_1
0 True
1 False
2 False
3 True


In [92]:
# oracle = useful_reads[9][1].iloc[0,2:-1]
# oracle2 = useful_reads[9][1].iloc[1,2:-1] <- Good ones 

oracle = useful_reads[14][1].iloc[0,2:-1]
oracle2 = useful_reads[14][1].iloc[-1,2:-1] #<- Plus anchor

# oracle = useful_reads[idx][1].iloc[7,2:-1]
# oracle2 = useful_reads[idx][1].iloc[5,2:-1]


In [93]:
len(oracle)

95

In [94]:
test1 = [item for item in groups if (np.array(item[0]) == np.array(oracle)).all()]
test2 = [item for item in groups if (np.array(item[0]) == np.array(oracle2)).all()]

In [95]:
test1[0][1]

,repeat_unit,strand,SNP_257,SNP_2701,SNP_6161,SNP_4111,SNP_4235,SNP_5874,SNP_5812,SNP_2636,...,SNP_3050,SNP_1952,SNP_2424,SNP_385,SNP_1898,SNP_1458,SNP_7249,SNP_7202,SNP_7301,SNP_890
104,16d50e8d-e5a7-4d4e-a5f5-f69edd626e2d_9,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152,2903071d-6bb6-4abf-b49d-64dc73d7a01a_1,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
241,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_9,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
245,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_13,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
401,5a3aeaca-385e-4a67-b5d4-1e5885c42090_4,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
449,655011bd-e4e9-446f-8748-0a1fee8720ec_9,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
752,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_5,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
807,cedcf397-5a33-4257-9213-4ee1f1f36ae4_1,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
828,d83b46e6-dc86-44b2-a3c3-27023adaa13b_1,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
852,e0f6e516-8f88-487b-840e-2c8fa8f0a4b1_3,+,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [96]:
test2[0][1]

,repeat_unit,strand,SNP_257,SNP_2701,SNP_6161,SNP_4111,SNP_4235,SNP_5874,SNP_5812,SNP_2636,...,SNP_3050,SNP_1952,SNP_2424,SNP_385,SNP_1898,SNP_1458,SNP_7249,SNP_7202,SNP_7301,SNP_890
240,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
244,3dbd974f-d3f7-4b63-9177-ebbb4cf8f7ca_12,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
414,5b4d129f-0d6c-4b8b-bf58-10cf5c8265f5_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
448,655011bd-e4e9-446f-8748-0a1fee8720ec_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
457,6572ce4a-e14d-4d11-bcd5-622528f66c14_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
751,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_4,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
851,e0f6e516-8f88-487b-840e-2c8fa8f0a4b1_2,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
962,f6394dc7-0754-4d9a-9343-03dca44c4d5a_3,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1009,fde056f2-a5a1-4132-9052-3c612088c885_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2198,16d50e8d-e5a7-4d4e-a5f5-f69edd626e2d_8,+,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
repeat_mappings[repeat_mappings[0].str.contains('bfe0a09d-bbc1-473d-bfc2-374db4a596b3')]

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,Seq,bam_strand,bam_cigar
748,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_1,7488,0,7488,+,C_briggsae_45S_rDNA,7493,0,7493,7482,...,60,tp:A:P,NM:i:15,mm:i:2,gn:i:13,go:i:6,308M2D3480M3D53M1D96M1I2661M3D4M3I882M,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,308M2D3480M3D53M1D96M1I2661M3D4M3I882M
749,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_2,7493,0,7493,+,C_briggsae_45S_rDNA,7493,0,7493,7481,...,60,tp:A:P,NM:i:16,mm:i:8,gn:i:8,go:i:6,308M2D253M1I3156M1I1420M2D1208M1I345M1I799M,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,308M2D253M1I3156M1I1420M2D1208M1I345M1I799M
750,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_3,7500,0,7500,+,C_briggsae_45S_rDNA,7493,0,7493,7479,...,60,tp:A:P,NM:i:25,mm:i:10,gn:i:15,go:i:11,247M1I61M2D102M1I10M1I193M4I28M1I100M1D1092M1I...,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,247M1I61M2D102M1I10M1I193M4I28M1I100M1D1092M1I...
751,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_4,7473,0,7473,+,C_briggsae_45S_rDNA,7493,0,7492,7460,...,60,tp:A:P,NM:i:34,mm:i:11,gn:i:23,go:i:18,270M1D37M2D342M1I667M1D1M1D527M1D1M2D440M2D29M...,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,270M1D37M2D342M1I667M1D1M1D527M1D1M2D440M2D29M...
752,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_5,7493,0,7493,+,C_briggsae_45S_rDNA,7493,0,7493,7477,...,60,tp:A:P,NM:i:24,mm:i:8,gn:i:16,go:i:10,239M2I69M2D69M1I5M3I178M1I1858M3D5M1I2M1D2564M...,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,239M2I69M2D69M1I5M3I178M1I1858M3D5M1I2M1D2564M...
753,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_6,5972,0,5972,+,C_briggsae_45S_rDNA,7493,0,5980,5962,...,60,tp:A:P,NM:i:24,mm:i:4,gn:i:20,go:i:11,296M1I12M2D477M4D1133M1I96M1D2M1I483M1I385M1D2...,"(G, T, T, T, C, C, C, T, T, T, T, T, T, T, T, ...",+,296M1I12M2D477M4D1133M1I96M1D2M1I483M1I385M1D2...
2241,bfe0a09d-bbc1-473d-bfc2-374db4a596b3_0,7140,0,7140,+,C_briggsae_45S_rDNA,7493,353,7493,7137,...,60,tp:A:P,NM:i:6,mm:i:0,gn:i:6,go:i:5,263M1D618M1D483M1I785M2I3502M1D1486M,"(A, T, G, A, T, C, T, A, C, A, T, T, T, A, A, ...",+,263M1D618M1D483M1I785M2I3502M1D1486M


In [ ]:
asdf = ['093114d6-3ba4-455c-996a-fa6c5be17e0c_7', '0d7d0ee8-29c0-4ff1-a82a-802366b550ba_1', '0d7d0ee8-29c0-4ff1-a82a-802366b550ba_3', '1c9e48d2-2ebf-4d37-98df-cfcdde60e723_1',
        '2446013c-6172-4c2b-9e53-d0b50f37b94f_3', '25ad9965-bd9b-44d1-b02f-f14e7c7e9247_2', '2c295ee1-a4de-41b5-bc43-dabcb1a659e1_1', '42463130-8f56-485d-824f-6f0bd9e5fbf1_2',
        '42f0a68e-4cb1-40d9-ab36-e3a37eebd445_2', '431f6d58-d013-42b8-8633-ab17b8205be1_6', '431f6d58-d013-42b8-8633-ab17b8205be1_8', '45ae9b8f-b811-4c61-853f-298d5407d256_10',
        '45ae9b8f-b811-4c61-853f-298d5407d256_6', '45ae9b8f-b811-4c61-853f-298d5407d256_8', '483aa9c9-156c-4307-8545-18e0369df92c_1', '6c144884-2dce-4d7f-82be-1fd2594a1bdd_1',
        '7b850513-1c3f-4494-85b1-4c2a022ec8e3_7', '7b850513-1c3f-4494-85b1-4c2a022ec8e3_9', '7c25a61d-3d89-48c0-a0a8-f3c3de550fe7_1', '7d6e4a89-d8da-48b3-8426-cc0c232f1c44_5',
        '7f28fefb-93ac-49ba-b210-69932f354a38_1', '8e7a75fb-240c-42f6-a4eb-9991c8e85d6c_1', '9e36f9ee-559f-4cad-90b1-813ab61db198_2', 'b0d1a534-37ee-4c0d-8531-bc8fb83dfb21_4',
        'ddab370d-63bf-42eb-82d9-f2fb5595ebc8_5', 'e7d45d9d-58d3-4894-8a7f-ec7b4f586e1f_5', 'fb2db8c6-ecaf-4477-9764-3a6b9e7b84a7_8']